In [12]:
!pip install python-Levenshtein

In [13]:
import librosa
import numpy as np
import pdb
import string
from Levenshtein import distance
 
import os
from collections import Counter
import tensorflow.keras as keras
import tensorflow as tf
import sklearn
import random
import itertools
import pickle

In [14]:
 
def wav2feat(wavfile):
    '''
    Input: audio wav file name
    Output: Magnitude spectrogram
    '''
    x, Fs = librosa.load(wavfile, sr=44100, mono=True) 
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)
 
def wavs2feat(wavfiles):
    '''
    Concatenate the audio files listed in wavfiles
    Input: list of audio wav file names
    Output: Magnitude spectrogram of concatenated wav
    '''
    x = []
    for wf in wavfiles:
        x1, Fs = librosa.load(wf, sr=44100, mono=True)
        x.append(x1)
    x = np.hstack(x)
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)
 
def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label
 
def editDistance(gt, est):
    '''both are lists of labels
    E.g. gt is "dog_bark-street_music-engine_idling"
    E.g. est is "street_music-engine_idling"
    '''
    gttokens = gt.split('-')
    esttokens = est.split('-')
    # Map token to char
    tokenset = list(set(gttokens+esttokens)) # ['dog_bark', 'siren', 'street_music', 'engine_idling']
    token_char = {}
    for i in range(len(tokenset)):
        token_char[tokenset[i]] = string.ascii_uppercase[i]  # {'dog_bark': 'A', 'siren': 'B', 'street_music': 'C', 'engine_idling': 'D'}
    # convert gt and est to strings
    gtstr = [token_char[t] for t in gttokens]
    gtstr = ''.join(gtstr)  # 'BCA'
    eststr = [token_char[t] for t in esttokens]
    eststr = ''.join(eststr)  # 
    # Compare
    editdist = distance(gtstr, eststr) # 1
    score = 1 - editdist/len(gtstr)
    return editdist, score
 
def evals(gtcsv, estcsv, taskid):
    gt_id_label = read_csv(gtcsv)
    est_id_label = read_csv(estcsv)
    score = 0
    for id in est_id_label:
        if taskid==1:
            if est_id_label[id] == gt_id_label[id]:
                score += 1
        elif taskid==2:
            _, ss = editDistance(gt_id_label[id], est_id_label[id])
            score += ss
        else:
            pdb.set_trace()
            assert False, ["taskid not correct; it is", taskid]
    avgScore = score/len(est_id_label)
    return avgScore

In [15]:
name_label_dict = read_csv('/content/drive/My Drive/AED/labels_train.csv')
labels_list = list(set(name_label_dict.values()))
train_set_dist = Counter(name_label_dict.values())
label_to_idx = {'jackhammer':0, 'drilling':1, 'dog_bark':2, 'air_conditioner':3, 'engine_idling':4, 'children_playing':5, 'siren':6, 'gun_shot':7, 'car_horn':8, 'street_music':9}
idx_to_label = {0:'jackhammer', 1:'drilling', 2:'dog_bark', 3:'air_conditioner', 4:'engine_idling', 5:'children_playing', 6:'siren', 7:'gun_shot', 8:'car_horn', 9:'street_music'}
print(labels_list, train_set_dist)

['children_playing', 'gun_shot', 'street_music', 'air_conditioner', 'car_horn', 'engine_idling', 'class', 'dog_bark', 'jackhammer', 'siren', 'drilling'] Counter({'jackhammer': 240, 'dog_bark': 200, 'drilling': 200, 'children_playing': 200, 'street_music': 200, 'air_conditioner': 200, 'engine_idling': 196, 'siren': 177, 'car_horn': 78, 'gun_shot': 70, 'class': 1})


In [ ]:
path = '/content/drive/My Drive/audio_train_1ch'
files=[]
files = [f for f in sorted(os.listdir(path))]
X_train_dict = {'jackhammer':[], 'drilling':[], 'dog_bark':[], 'air_conditioner':[], 'class':[], 'children_playing':[], 'siren':[], 'gun_shot':[], 'car_horn':[], 'street_music':[], 'engine_idling':[]}
 
for f in files:
  wavfile = [path + '/' + f]
  x = wavs2feat(wavfile)
  X_train_dict[name_label_dict[f]].append(x)
  if x.shape[1] != 401 or x.shape[0] != 513:
    print(f, x.shape, name_label_dict[f])

KeyboardInterrupt: ignored

In [ ]:
with open('/content/drive/My Drive/AED/X_train_dict', 'wb') as handle:
    pickle.dump(X_train_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

False

In [16]:
with open('/content/drive/My Drive/AED/X_train_dict', 'rb') as handle:
    X_train_dict = pickle.load(handle)

In [ ]:
(X['jackhammer'][2] == X_train_dict['jackhammer'][2]).all()

NameError: ignored

TASK 1

---



In [ ]:
X_train = []
targets = []
for a,b in X_train_dict.items():
  for c in b:
    X_train.append(c)
    targets.append(label_to_idx[a])

nb_classes = 10
targets = np.array(targets).reshape(-1)
Y_train = np.eye(nb_classes)[targets]

X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

print(len(X_train))

1408


In [ ]:
a = np.array([1,2,0])
np.argmax(a)

1

In [ ]:
def accuracy(true, pred):
  n = true.shape[0]
  correct = 0
  for i in range(n):
    if (np.argmax(true[i]) == np.argmax(pred[i]))  and pred[i][np.argmax(pred[i])] >= 0.5:
      correct = correct+1
  return correct/n

In [ ]:
def model():
  inp = keras.layers.Input(shape = (None, 513))
  out = tf.keras.layers.LSTM(100)(inp)
  out = tf.keras.layers.Dense(10, activation = 'softmax')(out)
  mod = tf.keras.Model(inp, out)
  return mod

In [ ]:
pred = model()
pred.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 513)]       0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               245600    
_________________________________________________________________
dense (Dense)                (None, 10)                1010      
Total params: 246,610
Trainable params: 246,610
Non-trainable params: 0
_________________________________________________________________


In [ ]:
pred = tf.keras.models.load_model('/content/drive/My Drive/AED/task1_mod.hdf5')

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 0.001)
cce = tf.keras.losses.CategoricalCrossentropy()
no_of_samples = len(X_train)
for i in range(1000):
  print('iteration ;:::: ', i)
  loss = 0
  predicted = []
  no_of_samples = len(X_train)
  with tf.GradientTape() as Tape:
    for k in range(no_of_samples):
      inp = X_train[k]
      inp = np.expand_dims(np.transpose(inp), axis = 0)
      out = np.expand_dims(Y_train[k], axis = 0)
      output_ = pred(inp)
      predicted.append(output_.numpy())
      loss_temp =  tf.reduce_mean(cce(out, output_))
      loss += loss_temp
      #print(out, output_, loss_temp)
    loss = loss/no_of_samples
  predicted = np.concatenate(predicted, axis = 0)
  print('TRAIN :;;;;;: ', accuracy(Y_train[0:no_of_samples,:], predicted), loss)
  grad = Tape.gradient(loss, pred.trainable_variables)
  optimizer.apply_gradients(zip(grad, pred.trainable_variables))

  ## validation
  loss = 0
  predicted = []
  no_of_samples = len(X_test)
  for k in range(no_of_samples):
    inp = X_test[k]
    inp = np.expand_dims(np.transpose(inp), axis = 0)
    out = np.expand_dims(Y_test[k], axis = 0)
    output_ = pred(inp)
    predicted.append(output_.numpy())
    loss_temp =  tf.reduce_mean(cce(out, output_))
    loss += loss_temp
    #print(out, output_, loss_temp)
  loss = loss/no_of_samples
  predicted = np.concatenate(predicted, axis = 0)
  print('TEST :;;;: ', accuracy(Y_test[0:no_of_samples,:], predicted), loss)


  # SAVE
  pred.save('/content/drive/My Drive/AED/task1_mod.hdf5')

iteration ;::::  0
TRAIN :;;;;;:  0.8295454545454546 tf.Tensor(0.4122141, shape=(), dtype=float32)
TEST :;;;:  0.6912181303116147 tf.Tensor(0.7706523, shape=(), dtype=float32)
iteration ;::::  1
TRAIN :;;;;;:  0.8352272727272727 tf.Tensor(0.4154482, shape=(), dtype=float32)
TEST :;;;:  0.7138810198300283 tf.Tensor(0.78344816, shape=(), dtype=float32)
iteration ;::::  2
TRAIN :;;;;;:  0.8288352272727273 tf.Tensor(0.45595387, shape=(), dtype=float32)
TEST :;;;:  0.7025495750708215 tf.Tensor(0.8007365, shape=(), dtype=float32)
iteration ;::::  3


KeyboardInterrupt: ignored

TASK 2

---





In [ ]:
N = 1000
X = []
Y = []
for i in range(N):
  n = random.randint(1, 5)
  classes = np.random.choice(range(10), n, replace=False)
  temp_x = []
  temp_y = []
  for j in classes:
    x =  random.sample(X_train_dict[idx_to_label[j]], 1)[0]
    temp_x.append(x)
    temp_y.extend([j]*x.shape[1])
  nb_classes = 10
  temp_y = np.array(temp_y).reshape(-1)
  temp_y = np.eye(nb_classes)[temp_y]
  Y.append(temp_y)
  temp_x = np.concatenate(temp_x, axis = 1)
  X.append(temp_x)


In [ ]:
for i in range(10):
  print(X[i].shape, Y[i][0], Y[i][1], Y[i][2])

(513, 401) [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
(513, 986) [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
(513, 1339) [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
(513, 1670) [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(513, 401) [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
(513, 802) [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
(513, 918) [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
(513, 1343) [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(513, 1203) [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
(513, 6) [0. 0. 0. 0. 0. 0. 0. 0.

In [ ]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.20, random_state=42)

In [ ]:
def model2():
  inp = keras.layers.Input(shape = (None, 513))
  out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True))(inp)
  out = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(100, activation = 'tanh'))(out)
  out = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(10, activation = 'softmax'))(out)

  mod = tf.keras.Model(inp, out)
  return mod

In [ ]:
pred2 = model2()
pred2.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 513)]       0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 200)         491200    
_________________________________________________________________
time_distributed (TimeDistri (None, None, 100)         20100     
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 10)          1010      
Total params: 512,310
Trainable params: 512,310
Non-trainable params: 0
_________________________________________________________________


In [ ]:
pred2 = tf.keras.models.load_model('/content/drive/My Drive/AED/task2_mod.hdf5')

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 0.001)
cce = tf.keras.losses.CategoricalCrossentropy()
no_of_samples = len(X_train)
for i in range(1000):
  print('iteration ;:::: ', i)
  loss = 0
  no_of_samples = len(X_train)
  with tf.GradientTape() as Tape:
    for k in range(no_of_samples):
      inp = X_train[k]
      inp = np.expand_dims(np.transpose(inp), axis = 0)
      out = np.expand_dims(Y_train[k], axis = 0)
      output_ = pred2(inp)
      loss_temp =  tf.reduce_mean(cce(out[0], output_[0]))
      loss += loss_temp
      #print(out, output_, loss_temp)
    loss = loss/no_of_samples
  print('TRAIN :;;;;;: ', loss)
  grad = Tape.gradient(loss, pred2.trainable_variables)
  optimizer.apply_gradients(zip(grad, pred2.trainable_variables))

  ## validation
  loss = 0
  no_of_samples = len(X_test)
  for k in range(no_of_samples):
    inp = X_test[k]
    inp = np.expand_dims(np.transpose(inp), axis = 0)
    out = np.expand_dims(Y_test[k], axis = 0)
    output_ = pred2(inp)
    loss_temp =  tf.reduce_mean(cce(out[0], output_[0]))
    loss += loss_temp
    #print(out, output_, loss_temp)
  loss = loss/no_of_samples
  print('TEST :;;;: ', loss)


  # SAVE
  pred2.save('/content/drive/My Drive/AED/task2_mod.hdf5')

iteration ;::::  0
TRAIN :;;;;;:  tf.Tensor(2.3688111, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(2.0778081, shape=(), dtype=float32)
iteration ;::::  1
TRAIN :;;;;;:  tf.Tensor(2.0644038, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(1.9379039, shape=(), dtype=float32)
iteration ;::::  2
TRAIN :;;;;;:  tf.Tensor(1.9082844, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(1.8293921, shape=(), dtype=float32)
iteration ;::::  3
TRAIN :;;;;;:  tf.Tensor(1.7968621, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(1.735201, shape=(), dtype=float32)
iteration ;::::  4
TRAIN :;;;;;:  tf.Tensor(1.7064688, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(1.6562036, shape=(), dtype=float32)
iteration ;::::  5
TRAIN :;;;;;:  tf.Tensor(1.6308461, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(1.5842907, shape=(), dtype=float32)
iteration ;::::  6
TRAIN :;;;;;:  tf.Tensor(1.560274, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(1.516459, shape=(), dtype=float32)
iteration ;::::  7
TRAIN :;;;;;:  tf.

KeyboardInterrupt: ignored

Another Fling

---






In [ ]:
## eval based on pred (bruteforce)
jump = 5
thresh = 0.7
for i in range(len(X_test)):
  x = X_test[i]
  x = np.expand_dims(np.transpose(x), axis = 0)
  y = []
  for j in range(Y_test[i].shape[0]):
    if np.argmax(Y_test[i][j]) in y:
      pass
    else:
      y.append(np.argmax(Y_test[i][j]))
  
  ts = x.shape[1]
  ans = []
  j = 0
  while j < ts:
    k = j+80
    while k < ts:
      output_ = pred(x[:,j:k,:]).numpy()[0]
      #print(j,k,':', output_)
      if output_[np.argmax(output_)] > thresh:
        ans.append(np.argmax(output_))
        j = k
        break
      k = k + jump
    j = j + jump
  print(y, ans)


NameError: ignored

In [ ]:
## eval based on pred2 
jump = 5
thresh = 0.8
for i in range(len(X_test)):
  x = X_test[i]
  x = np.expand_dims(np.transpose(x), axis = 0)
  y = []
  for j in range(Y_test[i].shape[0]):
    if np.argmax(Y_test[i][j]) in y:
      pass
    else:
      y.append(np.argmax(Y_test[i][j]))
  
  
  output_ = pred2(x).numpy()[0]
  for k in range(output_.shape[0]):
    ans.append(np.argmax(output_[k]))
  ans = [a[0] for a in itertools.groupby(ans) if list(a[1])[0]>5]
  print(y, ans)


TRIGGER DETECTOR

---

---





In [17]:
N = 1000
win = 50
X = []
Y = []
for i in range(N):
  n = random.randint(1, 5)
  classes = np.random.choice(range(10), n, replace=False)
  temp_x = []
  lengths = []
  for j in classes:
    x =  random.sample(X_train_dict[idx_to_label[j]], 1)[0]
    temp_x.append(x)
    lengths.append(x.shape[1])
  
  temp_x = np.concatenate(temp_x, axis = 1)
  X.append(temp_x)
 
  temp_y = np.zeros(temp_x.shape[1])
  sm = 0
  for j in range(len(lengths)-1):
    sm = sm+lengths[j]
    for k in range(max(0,sm-win), min(temp_x.shape[1], sm+win)):
      temp_y[k] = 1
  
  Y.append(temp_y)

In [18]:
del(X_train_dict)

In [19]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.20, random_state=42)

In [20]:
del(X)
del(Y)

In [21]:
out = np.expand_dims(np.expand_dims(Y_train[0], axis = 0), axis = -1)
for i in range(out.shape[1]):
  print(out.shape)

(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 401, 1)
(1, 

In [ ]:
def model3():
  inp = keras.layers.Input(shape = (None, 513))
  out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True))(inp)
  #out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True))(out)
  out = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(50, activation = 'relu'))(out)
  out = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation = 'sigmoid'))(out)

  mod = tf.keras.Model(inp, out)
  return mod

In [ ]:
pred3 = model3()
pred3.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 513)]       0         
_________________________________________________________________
bidirectional (Bidirectional (None, None, 200)         491200    
_________________________________________________________________
time_distributed (TimeDistri (None, None, 50)          10050     
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 1)           51        
Total params: 501,301
Trainable params: 501,301
Non-trainable params: 0
_________________________________________________________________


In [22]:
pred3 = tf.keras.models.load_model('/content/drive/My Drive/AED/task3_mod.hdf5')

In [25]:
optimizer = keras.optimizers.Adam(learning_rate = 0.001)
bce = tf.keras.losses.BinaryCrossentropy()
no_of_samples = len(X_train)
for i in range(1000):
  print('iteration ;:::: ', i)
  loss = 0
  no_of_samples = len(X_train)
  with tf.GradientTape() as Tape:
    for k in range(no_of_samples):
      inp = X_train[k]
      inp = np.expand_dims(np.transpose(inp), axis = 0)
      out = np.expand_dims(np.expand_dims(Y_train[k], axis = 0), axis = -1)
      output_ = pred3(inp)
      loss_temp =  tf.reduce_mean(bce(out[0], output_[0]))
      loss += loss_temp
      #print(out, output_, loss_temp)
    loss = loss/no_of_samples
  print('TRAIN :;;;;;: ', loss)
  grad = Tape.gradient(loss, pred3.trainable_variables)
  optimizer.apply_gradients(zip(grad, pred3.trainable_variables))
 
  ## validation
  loss = 0
  no_of_samples = len(X_test)
  for k in range(no_of_samples):
    inp = X_test[k]
    inp = np.expand_dims(np.transpose(inp), axis = 0)
    out = np.expand_dims(np.expand_dims(Y_test[k], axis = 0), axis = -1)
    output_ = pred3(inp)
    loss_temp =  tf.reduce_mean(bce(out[0], output_[0]))
    loss += loss_temp
    #print(out, output_, loss_temp)
  loss = loss/no_of_samples
  print('TEST :;;;: ', loss)
 
  # SAVE
  pred3.save('/content/drive/My Drive/AED/task3_mod.hdf5')

iteration ;::::  0
TRAIN :;;;;;:  tf.Tensor(0.0314569, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(1.0488776, shape=(), dtype=float32)
iteration ;::::  1
TRAIN :;;;;;:  tf.Tensor(0.80898285, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(0.30999595, shape=(), dtype=float32)
iteration ;::::  2
TRAIN :;;;;;:  tf.Tensor(0.15832348, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(0.6341457, shape=(), dtype=float32)
iteration ;::::  3
TRAIN :;;;;;:  tf.Tensor(0.49950433, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(0.6592377, shape=(), dtype=float32)
iteration ;::::  4
TRAIN :;;;;;:  tf.Tensor(0.53307253, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(0.52458155, shape=(), dtype=float32)
iteration ;::::  5
TRAIN :;;;;;:  tf.Tensor(0.36891845, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(0.36774844, shape=(), dtype=float32)
iteration ;::::  6
TRAIN :;;;;;:  tf.Tensor(0.21650028, shape=(), dtype=float32)
TEST :;;;:  tf.Tensor(0.28772095, shape=(), dtype=float32)
iteration ;::::  7
TRAIN

KeyboardInterrupt: ignored

In [ ]:
,
